In [1]:
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, GridspecLayout

In [2]:
# Image Widget

file = open("../logo.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='200'
                )

label_headline = widgets.Label(
                    value='',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [3]:
global stored_choices, path

# stored_choices = {}
path = "../temp_storage.json"
back_path = "../backtracking.json"

In [4]:
with open(path, "r") as f:
    stored_choices = json.load(f)

In [5]:
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [6]:
# Read the CSV file into a DataFrame

df = pd.read_csv('new_format.csv', low_memory=False)

In [7]:
notebook_list = ["objective", "num_cams", "cam_strat_covar", "surv_dur_min_max", "sp_asymptote", 
                 "obj_targ_sp", "sp_info", "sp_occ_restr", "sp_hr_size", "sp_size", "sp_rarity", 
                 "sp_detprob_cat", "cam_independent"]

In [8]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list

def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [9]:
# gets the question based on value of other column

ques1 = df.loc[df['question_code'] == 'data_hr', 'question_text'].iloc[0]
opt1 = df.loc[df['question_code'] == 'data_hr', 'q_option_label'].iloc[0]
opt_codes1 = df.loc[df['question_code'] == 'data_hr', 'q_option_code'].iloc[0]

opt1 = split_and_trim(opt1)
opt_codes1 = split_and_trim(opt_codes1)

ques2 = df.loc[df['question_code'] == 'hr_size', 'question_text'].iloc[0]

In [10]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto')) #width='max-content'

In [11]:
# Factor widget

ques_button = create_expanded_button(ques1, 'success')
ques_button.style.button_color = 'lightblue'
ques_button.style.text_color = 'black'

option_1_button = create_expanded_button(opt1[0], 'success')
option_1_button.style.button_color = 'lightgreen'
option_1_button.style.text_color = 'black'

option_2_button = create_expanded_button(opt1[1], 'success')
option_2_button.style.button_color = 'red'
option_2_button.style.text_color = 'black'

factor_grid = GridspecLayout(4, 12, grid_gap='5px', height='200px')
factor_grid[1:3, 0:6] = ques_button
factor_grid[1, 7:9] = option_1_button
factor_grid[2, 7:9] = option_2_button


# factor_grid

In [12]:
global options_widget2
options_widget2 = widgets.IntText(
                    value=0,  # Default value
                    description=ques2,
                    disabled=False,
                    layout={'width': 'max-content'}
                )

In [13]:
output = widgets.Output()
def option_1_button_clicked(event):
    with output:
        clear_output()
        global selected_objective, selected_target_sp
        display(options_widget2)
        
        #### Below code block doesen't get executed.................. solve this next
        
#         print("EEEE")
#         if options_widget2.value > 0:
#             stored_choices["sp_hr_size"] = options_widget2.value
#             options_widget2.value = 0
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)
            
#         selected_objective = stored_choices['objective']
#         selected_target_sp = stored_choices['obj_targ_sp']
# #         selected_sp_type = stored_choices['sp_type']

factor_grid[1, 7:9].on_click(option_1_button_clicked)

In [14]:
output = widgets.Output()
def option_2_button_clicked(event):
    with output:
        clear_output()
        global selected_objective, selected_target_sp
        
        selected_option = factor_grid[2, 7:9].description
        selected_option_index = opt1.index(selected_option)
        selected_option_code = opt_codes1[selected_option_index]
        
        stored_choices["sp_hr_size"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        selected_objective = stored_choices['objective']
        selected_target_sp = stored_choices['obj_targ_sp']
#         selected_sp_type = stored_choices['sp_type']
        
factor_grid[2, 7:9].on_click(option_2_button_clicked)

In [15]:
# Create a button for going back to previous question

back_button = widgets.Button(description="Back")

In [16]:
# Create a buttons for submitting the question

submit_button = widgets.Button(description="Submit")

In [17]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(link)

back_button.on_click(on_back_button_clicked)

In [18]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()
        global selected_option1
        
        link = "Submit your answer for next question"

        condition = df.loc[df['page_name'] == 'sp_size', 'json_logic'].iloc[0]
        condition_json = json.loads(condition)
        if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
            next_notebook = "sp_size"
        
        condition = df.loc[df['page_name'] == 'sp_rarity', 'json_logic'].iloc[0]
        condition_json = json.loads(condition)
        if (selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']):# or (selected_objective in condition_json['objective'] and selected_sp_type in condition_json['sp_type'])
            next_notebook = "sp_rarity"
        
        condition = df.loc[df['page_name'] == 'sp_detprob_cat', 'json_logic'].iloc[0]
        condition_json = json.loads(condition)
        if selected_objective in condition_json['objective'] and selected_target_sp in condition_json['obj_targ_sp']:
            next_notebook = "sp_detprob_cat"
        
        condition = df.loc[df['page_name'] == 'cam_independent', 'json_logic'].iloc[0]
        condition_json = json.loads(condition)
        if selected_objective in condition_json['objective']:
            next_notebook = "cam_independent"
        else:
            next_notebook = "response_summary"
            
            
        notebook_filename = next_notebook + ".ipynb"
        
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Next</a>'''.format(notebook_filename))

        back_list.append("sp_hr_size")
        with open (back_path, "w")as f:
            json.dump(back_list, f)

        display(link)

submit_button.on_click(on_button_clicked)

In [20]:
# Apply center alignment using CSS styling to the container

container = widgets.VBox(children=[factor_grid, output], layout=widgets.Layout(
    align_items='center'  # Center align vertically
))

button_container = widgets.HBox(children=[back_button, submit_button], layout=widgets.Layout(
    justify_content='center',  # Center align horizontally
))


display(vbox_headline)
display(container)
display(button_container)